## Process of scraping the website

In [1]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [8]:
np.arange(0, 3001, 120)

array([   0,  120,  240,  360,  480,  600,  720,  840,  960, 1080, 1200,
       1320, 1440, 1560, 1680, 1800, 1920, 2040, 2160, 2280, 2400, 2520,
       2640, 2760, 2880, 3000])

In [23]:
def get_url(city_name, num_post, main_city=True):
    if main_city:
        sub_city = city_name[0:3] + '/'
    return f'https://{city_name.lower()}.craigslist.org/search/{sub_city.lower()}apa?s={num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
# Chilliwack rental housing

response = get(get_url(city_name = 'Vancouver', num_post = 2880))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
129


In [24]:
# Show first post to look into it
post_one = posts[-1:]
post_one

[<li class="result-row" data-pid="7559600963">
 <a class="result-image gallery" data-ids="3:00x0x_cOMzjF065vtz_0kE0dM,3:00O0O_no92ZZLQxSz_0kE0dM,3:00F0F_3yF8s43emeSz_0kE0dM,3:00o0o_9jbQTvBaa1Mz_0kE0dM,3:00E0E_kjJSCIgBMuyz_0kE0dM,3:00O0O_bKPhf6TPMJyz_0kE0dM,3:00v0v_lKFvwVNuQlYz_0kE0dM,3:00J0J_6JDVZvpMoz0z_0kE0dM,3:00d0d_gPrlxRnLKc1z_0kE0dM,3:00a0a_8ceRkdz1nqXz_0kE0dM,3:00303_dUU6pg7hkzEz_0oc0g6,3:00B0B_8QqpDBOyRlSz_0CI0pP" href="https://vancouver.craigslist.org/van/apa/d/vancouver-vancouver-yaletown-ocean-view/7559600963.html">
 <span class="result-price">$4,380</span>
 </a>
 <div class="result-info">
 <span class="icon icon-star" role="button">
 <span class="screen-reader-text">favorite this post</span>
 </span>
 <time class="result-date" datetime="2022-11-20 08:51" title="Sun 20 Nov 08:51:32 AM">Nov 20</time>
 <h3 class="result-heading">
 <a class="result-title hdrlnk" data-id="7559600963" href="https://vancouver.craigslist.org/van/apa/d/vancouver-vancouver-yaletown-ocean-view/7559600

In [4]:
# City posted
post_city = post_one.find('span', class_='result-hood').text.strip('( )')
post_city

'Abbotsford'

In [5]:
# Number of bedroom
num_bed = int(post_one.find('span', class_='housing').text.split()[0].replace('br', ''))
num_bed, type(num_bed)

(1, int)

In [6]:
# SQFT
sqft = int(post_one.find('span', class_='housing').text.split()[2].replace('ft2', ''))
sqft, type(sqft)

(900, int)

In [7]:
# rental price
price = int(post_one.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
price, type(price)

(1850, int)

In [8]:
# Post ad title
post_ad = post_one.find('a', class_='result-title hdrlnk').text
post_ad

'Luxury Loft 1 bed + Den'

In [9]:
# post datetime
datetime = datetime.strptime(post_one.find('time', class_='result-date')['datetime'], '%Y-%m-%d %H:%M')
datetime, type(datetime)

(datetime.datetime(2022, 12, 6, 16, 44), datetime.datetime)

In [10]:
search_legend = soup.find('div', class_='search-legend')
search_legend

<div class="search-legend">
<div class="search-view js-only">
<div aria-expanded="false" aria-label="view options" class="dropdown dropdown-icons dropdown-arrows dropdown-view" role="toolbar">
<ul class="dropdown-list">
<li class="dropdown-item mode sel">
<button data-selection="list" id="listview" title="show results in a list">
<span class="view-list icon"></span>list
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="pic" id="picview" title="show results in a list with thumbnail pictures">
<span class="view-thumb icon"></span>thumb
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="grid" id="gridview" title="show results side-by-side with larger pictures">
<span class="view-gallery icon"></span>gallery
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="map" id="mapview" 

In [11]:
# Find out the beggining of the post number 
num_post_start = search_legend.find('span', class_='rangeFrom').text
num_post_start

'121'

In [12]:
# Find out total number of the search result
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
results_total

1532